In [1]:
# O objetivo final é identificar os algarismos na foto, gerar uma imagem quadrada
# de tamanho padrão com cada um deles centralizados e retornar os vetores que
# representam cada imagem.
#
# Atenção: quando a imagem aparecer não feche a janela, aperte alguma tecla ou então
# o jupyter da erro.

In [2]:
import numpy as np
import cv2

In [3]:
# deixa a imagem em escala de cinza e borra para reduzir o número de contornos
img = cv2.imread('teste2.jpeg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray,(5,5),0)

In [4]:
# binariza
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 135, 5)

In [5]:
# identifica contornos na imagem
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

In [6]:
# calcula as áreas dos retângulos construídos pelos contornos em contours
# o retângulo é limitado entre x e x+h na horizontal e entre y e y+w na vertical
area = np.zeros(len(contours))
for i, ctr in enumerate(contours):
    x, y, w, h = cv2.boundingRect(ctr)
    area[i] = w*h
bigarea = np.amax(area[0:-1])
# bigarea é a area do maior dígito, area[-1] corresponde a toda a imagem e por isso é desprezado

In [7]:
# imaginando que os algarismos são os maiores contornos, é suposto que
# todo contorno de tamanho superior a 15% do maior corresponde a um algarismo

j = 1
for i, ctr in enumerate(contours):
    x, y, w, h = cv2.boundingRect(ctr)
    if 0.15*bigarea < area[i]:
        cv2.imwrite("test_results/"+str(j)+".png",thresh[y:y+h,x:x+w])
        j += 1

In [36]:
# adiciona borda branca
Vetores = []

for i in range(1,j-1):
    img = cv2.imread("test_results/"+str(i)+".png")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    m, n = (np.size(img,0),np.size(img,1))
    M = 255*np.ones((int(m+0.2*m), int(m + 0.2*m)))#, dtype = np.int8)
    M[int(0.1*m + 1): int(0.1*m + m) + 1, int((0.1*m + 0.5*m - 0.5*n) + 1): int((0.1*m + 0.5*m - 0.5*n) + n) + 1] = img    
    
    # caso queira ir vendo
    cv2.imshow('img', M)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    M = cv2.resize(M,dsize = (8,8))
    Vetores.append(M.flatten())
    
    # caso queira ir vendo
    cv2.imshow('img', M)
    cv2.waitKey(0)
    cv2.destroyAllWindows()